Data preparing for neural network training. We will use the average performance of a team of two previous years' TourneyCompactResult and current year's RegularSeasonDetailedResults as X_train, plus two features: previous two years' Tourney match number and current year's win_rate. The train data format is h(win_team_detail_result + lose_team_detail_result) = 1 and h(lose_team_detail_result + win_team_detail_result) = 0.
To predict 2017's result, we will need 2017's regular seson detail result.

In [7]:
import sys
import numpy as np
import pandas as pd
import matplotlib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import roc_curve, auc
#from sklearn import metrics
#from sklearn.cross_validation import train_test_split


In [9]:
data = {}
csv = ['/RegularSeasonCompactResults.csv',
'/RegularSeasonDetailedResults.csv',
'/Seasons.csv',
'/Teams.csv',
'/TourneyCompactResults.csv',
'/TourneyDetailedResults.csv',
'/TourneySeeds.csv',
'/TourneySlots.csv',
'/sample_submission.csv']
path = "/Users/wangliran/Desktop/kaggle/March-Machine-Learning-Mania-2017/data"
for file in csv:
    data[file[1:-4]] = pd.read_csv(path+file)
    print(data[file[1:-4]].head(5))
# df_seeds = pd.read_csv(path + csv[6])
# df_tour = pd.read_csv(path + csv[4])
#print(data["TourneySeeds"].head())

# seed = data["TourneySeeds"]
# seed["Wteam"] = seed["Team"]


   Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot
0    1985      20   1228      81   1328      64    N      0
1    1985      25   1106      77   1354      70    H      0
2    1985      25   1112      63   1223      56    H      0
3    1985      25   1165      70   1432      54    H      0
4    1985      25   1192      86   1447      74    H      0
   Season  Daynum  Wteam  Wscore  Lteam  Lscore Wloc  Numot  Wfgm  Wfga ...   \
0    2003      10   1104      68   1328      62    N      0    27    58 ...    
1    2003      10   1272      70   1393      63    N      0    26    62 ...    
2    2003      11   1266      73   1437      61    N      0    24    58 ...    
3    2003      11   1296      56   1457      50    N      0    18    38 ...    
4    2003      11   1400      77   1208      71    N      0    30    61 ...    

   Lfga3  Lftm  Lfta  Lor  Ldr  Last  Lto  Lstl  Lblk  Lpf  
0     10    16    22   10   22     8   18     9     2   20  
1     24     9    20   20   25     7 

Notice that compact result ranges from 1985-2016, while detailed result only ranges from 2003-2016. We will use the detailed result data, so we start from 2003.

Our aim: 1. get regular season win_rate
         2. get tourney season match number

In [42]:
def get_regular_win_rate(team, season): #int, int
    d = data["RegularSeasonCompactResults"][data["RegularSeasonCompactResults"]["Season"] == season]
    team_season_filter = (d["Wteam"] == team) | (d["Lteam"] == team) 
    team_win_filter = (d["Wteam"] == team) 
    #print d[team_season_filter]
    return float(d[team_win_filter].shape[0])/d[team_season_filter].shape[0]
get_regular_win_rate(1272, 2006)

0.9090909090909091

In [58]:
def get_match_number(team, season):
    d = data["TourneyCompactResults"][data["TourneyCompactResults"]["Season"] == season]
    team_season_filter = (d["Wteam"] == team) | (d["Lteam"] == team) 
    return d[team_season_filter].shape[0]
get_match_number(1272, 2006)

4

Now we consider the average performance of each team in season_list

In [173]:
def get_team_detail(tp, team, season_list):
    if tp == "regular":
        d = data["RegularSeasonDetailedResults"]
    elif tp == "tourney":
        d = data["TourneyDetailedResults"]
    team_win_filter = (d["Wteam"] == team) & (d["Season"]>= season_list[0]) & (d["Season"]<= season_list[-1])
    team_lose_filter = (d["Lteam"] == team) & (d["Season"]>= season_list[0]) & (d["Season"]<= season_list[-1])
    win_detail_col = ["Wteam", "Wscore", "Wfgm", "Wfga", "Wfgm3", "Wfga3", "Wftm", "Wfta",
                     "Wor", "Wdr", "Wast", "Wto", "Wstl", "Wblk", "Wpf"]
    lose_detail_col = ["Lteam", "Lscore", "Lfgm", "Lfga", "Lfgm3", "Lfga3", "Lftm", "Lfta",
                      "Lor", "Ldr", "Last", "Lto", "Lstl", "Lblk", "Lpf"]
    d1 = pd.DataFrame()
    combined1 = d[team_win_filter][win_detail_col]
    combined2 = d[team_lose_filter][lose_detail_col]
    d1[win_detail_col] = combined2[lose_detail_col]
    #combined = combined1.append(combined2)
    combined = combined1.append(d1)
    print(combined)
    temp1 = 0
    temp2 = 0
    for year in season_list:
        temp1 += get_regular_win_rate(team, year)
        temp2 += get_match_number(team, year)
    combined["win_rate"] = temp1/len(season_list)
    combined["game_number"] = temp2/len(season_list)
    return combined
a = get_team_detail("tourney", 1272, [2004,2005,2006])
b = get_team_detail("regular", 1272, [2006])
c = a.append(b)
d = c.apply(np.mean)
d["Season"] = 2006
print(d)


     Wteam  Wscore  Wfgm  Wfga  Wfgm3  Wfga3  Wftm  Wfta  Wor  Wdr  Wast  Wto  \
87    1272      59    23    59      9     27     4     7   14   22    16    9   
216   1272      94    37    61     11     22     9    16   12   21    16   13   
237   1272      72    23    48      7     16    19    26    7   18    15    9   
242   1272      80    31    60      6     15    12    18   11   30    11   13   
108   1272      53    20    50      6     17     7    14   11   11     9   12   
250   1272      45    17    54      2     17     9    15   16   25     5   18   

     Wstl  Wblk  Wpf  
87      3     4   15  
216    10     2   21  
237    10     2   20  
242    10     7   18  
108     6     3   15  
250     8     1   29  
       Wteam  Wscore  Wfgm  Wfga  Wfgm3  Wfga3  Wftm  Wfta  Wor  Wdr  Wast  \
13919   1272      79    27    61      6     21    19    26   10   35    17   
13936   1272      87    33    70      5     14    16    26   15   28    14   
14209   1272      88    31    58     

In [170]:
# def get_seed_num(season, team):
#     d = data["TourneySeeds"]
#     filter = (d["Team"] == team) & (d["Season"] == season)
#     return d[filter]["Seed"]
# get_seed_num(2006, 1272)
def seed_to_int(seed):
    """Get just the digits from the seeding. Return as int"""
    s_int = int(seed[1:3])
    return s_int
# data["TourneySeeds"]["n_seed"] = data["TourneySeeds"]["Seed"].apply(seed_to_int, axis=1)
# data["TourneySeeds"].drop(labels=['Seed'], inplace=True, axis=1)
# print(data["TourneySeeds"].head())
print(data["TourneySeeds"].head())

   Season  Team  n_seed
0    1985  1207       1
1    1985  1210       2
2    1985  1228       3
3    1985  1260       4
4    1985  1374       5


Prepare match detail for each seed starting from 2003, with n years' tourney, m years' regular match detail.

In [171]:
n = 2
m = 1
season_filter = data["TourneySeeds"]["Season"] >= 2003+n
seed_team = data["TourneySeeds"][season_filter]
print(seed_team.head())
print(seed_team.shape)
print(seed_team.iloc[0])
team_detail = pd.DataFrame()
for i in range(seed_team.shape[0]):
    team = seed_team.iloc[i]["Team"]
    year = seed_team.iloc[i]["Season"]
    a = get_team_detail("tourney", team, range(year-n, year))
    b = get_team_detail("regular", team, range(year+1-m, year+1))
    c = a.append(b)
    d = c.apply(np.mean)
    d["Season"] = year
    team_detail = team_detail.append(d, ignore_index=True)
print(team_detail.head())

      Season  Team  n_seed
1284    2005  1449       1
1285    2005  1448       2
1286    2005  1211       3
1287    2005  1257       4
1288    2005  1210       5
(798, 3)
Season    2005
Team      1449
n_seed       1
Name: 1284, dtype: int64
   Season       Wast      Wblk        Wdr       Wfga      Wfga3       Wfgm  \
0  2005.0  19.343750  2.687500  22.718750  65.062500  19.125000  31.625000   
1  2005.0  15.444444  3.694444  25.583333  58.805556  18.777778  28.777778   
2  2005.0  15.727273  3.424242  26.606061  55.575758  13.727273  27.606061   
3  2005.0  15.970588  3.794118  25.882353  57.588235  24.176471  26.794118   
4  2005.0  14.527778  4.888889  26.638889  58.694444  17.972222  26.444444   

      Wfgm3       Wfta       Wftm        Wor        Wpf     Wscore      Wstl  \
0  7.406250  21.875000  16.000000  14.906250  20.968750  86.656250  8.281250   
1  7.500000  26.833333  18.472222  13.555556  19.333333  83.527778  8.333333   
2  5.121212  26.000000  19.303030  11.909091  17.6

In [172]:
team_season_detail = team_detail
print(team_season_detail.columns.tolist())

['Season', 'Wast', 'Wblk', 'Wdr', 'Wfga', 'Wfga3', 'Wfgm', 'Wfgm3', 'Wfta', 'Wftm', 'Wor', 'Wpf', 'Wscore', 'Wstl', 'Wteam', 'Wto', 'game_number', 'win_rate']


Now we will prepare training and testing data. 

In [198]:
d = data["TourneyDetailedResults"]
d = d[d["Season"]>=2003+n]
print(d.iloc[0]["Season"])

win_detail_col = ['Wast', 'Wblk', 'Wdr', 'Wfga', 'Wfga3', 'Wfgm', 'Wfgm3', 'Wfta',
                  'Wftm', 'Wor', 'Wpf', 'Wscore', 'Wstl', 'Wto', 'game_number', 'win_rate']
lose_detail_col = ['Last', 'Lblk', 'Ldr', 'Lfga', 'Lfga3', 'Lfgm', 'Lfgm3', 'Lfta',
                  'Lftm', 'Lor', 'Lpf', 'Lscore', 'Lstl', 'Lto', 'Lgame_number', 'Lwin_rate']
w_filter = (team_detail["Season"] == d.iloc[0]["Season"]) & (team_detail["Wteam"]
                                                                     == d.iloc[0]["Wteam"])
l_filter = (team_detail["Season"] == d.iloc[0]["Season"]) & (team_detail["Wteam"]
                                                                     == d.iloc[0]["Lteam"])
wteam_detail = team_detail[w_filter][win_detail_col]
lteam_detail = team_detail[l_filter][win_detail_col]
wteam_detail[lose_detail_col] = lteam_detail[win_detail_col]
l = pd.DataFrame()
l[lose_detail_col] = lteam_detail[win_detail_col]
wteam_detail[lose_detail_col] = lteam_detail[win_detail_col]
# l1 = pd.concat([wteam_detail[win_detail_col], l], axis = 0)
l1 = wteam_detail[win_detail_col].append(l)

print(l1)
def prepare_train_test():
    train_test = pd.DataFrame
    for i in range(d.shape[0]):
        w_filter = (team_detail["Season"] == d.iloc[i]["Season"]) & (team_detail["Wteam"]
                                                                     == d.iloc[i]["Wteam"])
        l_filter = (team_detail["Season"] == d.iloc[i]["Season"]) & (team_detail["Wteam"]
                                                                     == d.iloc[i]["Lteam"])
        wteam_detail = team_detail[w_filter]
        lteam_detail = team_detail[l_filter]
        l_detail[lose_detail_col] = lteam_detail[win_detail_col] 
        
        result1[win_detail_col] = wteam_detail
        result1[lose_detail_col] = lteam_detail
        #result1 = pd.concat([wteam_detail, lteam_detail], axis=1)
        result1["label"] = 1
        #result2[win_detail_col]
        
        
        
        

2005
         Last      Lblk        Ldr  Lfga      Lfga3       Lfgm     Lfgm3  \
64        NaN       NaN        NaN   NaN        NaN        NaN       NaN   
63  13.137931  3.862069  24.310345  59.0  20.241379  23.655172  6.103448   

         Lfta       Lftm  Lgame_number    ...        Wfgm3       Wfta  \
64        NaN        NaN           NaN    ...     6.275862  21.034483   
63  25.517241  15.655172           1.0    ...          NaN        NaN   

         Wftm        Wor        Wpf     Wscore      Wstl        Wto  \
64  13.931034  11.862069  17.655172  70.344828  6.965517  12.034483   
63        NaN        NaN        NaN        NaN       NaN        NaN   

    game_number  win_rate  
64          2.0   0.37931  
63          NaN       NaN  

[2 rows x 32 columns]


Reading the data: season + daynum + team as unique key.
